In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import os
import math
import time

In [ ]:
topLeft = [77.05655524877061,14.337577790560069]
topRight = [77.06363628056992,14.337577790560069]
bottomLeft = [77.06363628056992,14.344126337407861]
bottomRight = [77.05655524877061,14.344126337407861]

In [ ]:
def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def getCoords(zoomLevel):
    topleft = deg2num(topLeft[1], topLeft[0], zoomLevel)  # (c,b)
    topright = deg2num(topRight[1], topRight[0], zoomLevel)  # (a,b)
    bottomright = deg2num(bottomRight[1], bottomRight[0], zoomLevel)  # (a,d)
    bottomleft = deg2num(bottomLeft[1], bottomLeft[0], zoomLevel)  # (c,d)
    xmin = min(topleft[0], topright[0], bottomleft[0], bottomright[0])
    xmax = max(topleft[0], topright[0], bottomleft[0], bottomright[0])
    ymin = min(topleft[1], topright[1], bottomleft[1], bottomright[1])
    ymax = max(topleft[1], topright[1], bottomleft[1], bottomright[1])
    return (xmin, xmax, ymin, ymax)

def download_map_tiles(base_url, output_folder, zoom_level, scale):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get start time
    start_time = time.time()

    # Iterate over each tile within the specified range
    xmin, xmax, ymin, ymax = getCoords(zoom_level)
    for x in range(xmin, xmax + 1):
        for y in range(ymin, ymax + 1):
            # Construct the URL for the current tile with scale=3 for 640x640
            tile_url = f"{base_url}&x={x}&y={y}&z={zoom_level}&scale={scale}"
            print(tile_url)
            try:
                # Send HTTP GET request to fetch the tile
                response = requests.get(tile_url)

                # Check if request was successful (status code 200)
                if response.status_code == 200:
                    # Save the tile image to a file in the output folder
                    filename = f"tile_{zoom_level}_{x}_{y}.png"
                    filepath = os.path.join(output_folder, filename)
                    with open(filepath, "wb") as f:
                        f.write(response.content)
                    print(f"Downloaded: {filename}")
                else:
                    print(f"Failed to download tile ({x}, {y}), HTTP status code: {response.status_code}")

            except Exception as e:
                print(f"Error downloading tile ({x}, {y}): {e}")

    # Get end time
    end_time = time.time()

    # Print the total execution time
    print(f"Total time taken: {end_time - start_time} seconds")

In [ ]:
base_url = "https://mt1.google.com/vt/lyrs=s"
zoom_level = 17       # Zoom level
scale = 1           # scale of 1 = 256*256 dimensional image

output_folder = "/content/drive/MyDrive/Ponds_Wells_Plantations_Detection/Data_Download/Data/Zoom17/3TRY"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created the folder: {output_folder}")
else:
    print(f"The folder already exists: {output_folder}")


Created the folder: /content/drive/MyDrive/Pond/Data_Download/Final/Zoom17/3TRY


In [ ]:
download_map_tiles(base_url, output_folder, zoom_level, scale)

https://mt1.google.com/vt/lyrs=s&x=93591&y=60258&z=17&scale=1
Downloaded: tile_17_93591_60258.png
https://mt1.google.com/vt/lyrs=s&x=93591&y=60259&z=17&scale=1
Downloaded: tile_17_93591_60259.png
https://mt1.google.com/vt/lyrs=s&x=93591&y=60260&z=17&scale=1
Downloaded: tile_17_93591_60260.png
https://mt1.google.com/vt/lyrs=s&x=93592&y=60258&z=17&scale=1
Downloaded: tile_17_93592_60258.png
https://mt1.google.com/vt/lyrs=s&x=93592&y=60259&z=17&scale=1
Downloaded: tile_17_93592_60259.png
https://mt1.google.com/vt/lyrs=s&x=93592&y=60260&z=17&scale=1
Downloaded: tile_17_93592_60260.png
https://mt1.google.com/vt/lyrs=s&x=93593&y=60258&z=17&scale=1
Downloaded: tile_17_93593_60258.png
https://mt1.google.com/vt/lyrs=s&x=93593&y=60259&z=17&scale=1
Downloaded: tile_17_93593_60259.png
https://mt1.google.com/vt/lyrs=s&x=93593&y=60260&z=17&scale=1
Downloaded: tile_17_93593_60260.png
https://mt1.google.com/vt/lyrs=s&x=93594&y=60258&z=17&scale=1
Downloaded: tile_17_93594_60258.png
https://mt1.google.c

# Count Images in a folder

In [ ]:

import os

def count_images_in_folder(folder_path):
    # Define image file extensions you want to count
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff')

    # Initialize the image counter
    image_count = 0

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(image_extensions):
            image_count += 1

    return image_count

# Example usage:
folder_path = "/content/drive/MyDrive/Ponds_Wells_Plantations_Detection/Data_Download/Data/Zoom17/3TRY"
total_images = count_images_in_folder(folder_path)
print(f"Total images in the folder: {total_images}")


Total images in the folder: 1872


# Download Images by xtile and ytile


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import os
import time

# Function to download a specific map tile based on x and y coordinates
def download_map_tile(base_url, output_folder, zoom_level, x_tile, y_tile):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get start time
    start_time = time.time()

    # Construct the URL for the current tile
    tile_url = f"{base_url}&x={x_tile}&y={y_tile}&z={zoom_level}"
    print(tile_url)

    try:
        # Send HTTP GET request to fetch the tile
        response = requests.get(tile_url)

        # Check if request was successful (status code 200)
        if response.status_code == 200:
            # Save the tile image to a file in the output folder
            filename = f"tile_{zoom_level}_{x_tile}_{y_tile}.png"
            filepath = os.path.join(output_folder, filename)
            with open(filepath, "wb") as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
        else:
            print(f"Failed to download tile ({x_tile}, {y_tile}), HTTP status code: {response.status_code}")

    except Exception as e:
        print(f"Error downloading tile ({x_tile}, {y_tile}): {e}")

    # Get end time
    end_time = time.time()

    # Print the total execution time
    print(f"Total time taken: {end_time - start_time} seconds")

In [ ]:
# URL for Google map tiles (satellite imagery)
base_url = "https://mt1.google.com/vt/lyrs=s"
zoom_level = 18  # Set the zoom level

In [ ]:
# Specify the folder where you want to save the downloaded tiles
output_folder = "/content/drive/MyDrive/Ponds_Wells_Plantations_Detection/Data_Download/Data/Zoom17/3TRY"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created the folder: {output_folder}")
else:
    print(f"The folder already exists: {output_folder}")


Created the folder: /content/drive/MyDrive/Pond/Data_Download/tempWells


In [ ]:
# Manually input the x_tile and y_tile values for download
x_tile = 184111  # Replace with your x_tile value
y_tile = 112363 # Replace with your y_tile value

# Download the specific tile
download_map_tile(base_url, output_folder, zoom_level, x_tile, y_tile)

https://mt1.google.com/vt/lyrs=s&x=184111&y=112363&z=18
Downloaded: tile_18_184111_112363.png
Total time taken: 0.13039112091064453 seconds
